In [16]:
# ## KAGGLE ONLY
# from shutil import copyfile
# copyfile(src="../input/inputs/generate_dataloaders.py", dst="../working/generate_dataloaders.py")
# copyfile(src="../input/inputs/train_dataloader.p", dst="../working/train_dataloader.p")
# copyfile(src="../input/inputs/val_dataloader.p", dst="../working/val_dataloader.p")
# copyfile(src="../input/inputs/centroids_dataloader.p", dst="../working/ground_truth_dataloader.p")
# copyfile(src="../input/inputs/dictionary.p", dst="../working/dictionary.p")

# copyfile(src="../input/input2/train_unlabeld_dataloader.p", dst="../working/train_unlabelled_dataloader.p")
# copyfile(src="../input/input2/train_labeled_dataloader.p", dst="../working/train_labelled_dataloader.p")

In [17]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os
import datetime as dt
import pandas as pd
import random

from generate_dataloaders import *
from tqdm import tqdm_notebook as tqdm

import evaluation
import importlib
importlib.reload(evaluation)

<module 'evaluation' from '/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/evaluation.py'>

## Get Dataloaders

In [18]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [19]:
path = os.getcwd()
data_dir = path + '/data/'

In [20]:
train_loader = pkl.load(open(data_dir + 'train_dataloader.p','rb'))
train_loader_labelled = pkl.load(open(data_dir + 'train_labeled_dataloader.p','rb'))
train_loader_unlabelled = pkl.load(open(data_dir + 'train_unlabeled_dataloader.p','rb'))
val_loader = pkl.load(open(data_dir + 'val_dataloader.p','rb'))

In [21]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [22]:
print(torch.__version__)

1.3.1


## Neural Network Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [23]:
class neuralNetBow(nn.Module):
    """
    BagOfWords classification model
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, opts):
        super(neuralNetBow, self).__init__()
        self.embed = nn.Embedding(opts['vocab_size'], opts['emb_dim'], padding_idx=0)
        self.upweight = opts['upweight']
    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
        
        # upweight by flagged_index
        embedding[torch.LongTensor(range(batch_size)),flagged_index.type(torch.LongTensor),:] *= self.upweight
        
        # average across embeddings
        embedding_ave = embedding.sum(1) / (num_tokens + self.upweight - 1)
        
        return embedding_ave

### Clustering Stuff

In [24]:
class KMeansCriterion(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, embeddings, centroids):
        distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
        cluster_distances, cluster_assignments = distances.min(1)
        loss = cluster_distances.sum()
        return loss, cluster_assignments

In [25]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (tokens, labels, flagged_indices) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(tokens.to(current_device),flagged_indices.to(current_device))
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed.to(current_device))
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)
    
    #np_cluster_assignments = cluster_assignments.to('cpu')
    #np_counts = np.bincount(np_cluster_assignments.data.numpy(), minlength=k)
    #centroid_counts.add_(torch.FloatTensor(np_counts))

## Training Function

In [26]:
def train_model(model, centroids, criterion, optimizer, train_loader, valid_loader, num_epochs=10, path_to_save=None, print_every = 1000):

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    for epoch in range(num_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0

        # run one epoch of gradient descent on autoencoders wrt centroids
        for i, (tokens, labels, flagged_indices) in tqdm(enumerate(train_loader)):
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)

            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids.detach())

            # run update step
            optimizer.zero_grad()
            cluster_loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += cluster_loss.data

            # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments, sentence_embed)

            if i % print_every == 0:
                losses = cluster_loss.data/len(tokens)
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= len(train_loader.dataset)
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            torch.save(model.state_dict(), path_to_save+'model_dict.pt')
            torch.save(centroids, path_to_save+'centroids')
            torch.save(train_losses, path_to_save+'train_losses')
            torch.save(val_losses, path_to_save+'val_losses')
            torch.save(opts, path_to_save+'opts') #change options depending on model inputs required
        
    return model, centroids, train_losses, val_losses

In [27]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'
print("Current Device:",current_device)

Current Device: cpu


## Hyperparameter Tuning Loop

In [37]:
def get_save_directory(opts):
    path = os.getcwd()
    model_folder = 'baseline_randomized_embeddings/'
    model_dir = path + '/models/' + model_folder
    
    # subfolder for each hyperparam config
    emb_dim = opts['emb_dim']
    upweight = opts['upweight']
    subfolder = "emb_dim="+str(emb_dim) + ",upweight="+str(upweight) + '/'
    
    # need to actually create these subfolders lol
    try:
        os.makedirs(model_dir + subfolder) # will throw error if subfolder already exists
    except:
        pass
    
    return model_dir + subfolder

In [38]:
def train_config(opts):
    model = neuralNetBow(opts).to(current_device)
    centroids = centroid_init(2, opts['emb_dim'],train_loader_labelled, model, current_device)
    criterion = KMeansCriterion().to(current_device)
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    path_to_save = get_save_directory(opts)
    print(path_to_save)
    
    train_model(model, centroids, criterion, optimizer, train_loader, val_loader, num_epochs=10, path_to_save=path_to_save)

In [39]:
#emb_dims = [256, 512, 1024]
emb_dims = [512, 1024]
upweights = [1, 5, 10, 25]

for emb_dim in emb_dims:
    for upweight in upweights:
        opts = {
            'vocab_size': 20000,
            'emb_dim': emb_dim,
            'upweight': upweight
        }
        train_config(opts)

/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/emb_dim=256,upweight=1/
2019-12-05 00:50:43.401612 | Epoch 0


Average training loss at batch  0 : 2.487
Average training loss at batch  1000 : 0.218
Average training loss at batch  2000 : 0.127
Average training loss at batch  3000 : 0.148

Average training loss after epoch  0 : 0.364
Average validation loss after epoch  0 : 0.121
2019-12-05 00:53:15.382962 | Epoch 1


Average training loss at batch  0 : 0.095
Average training loss at batch  1000 : 0.059
Average training loss at batch  2000 : 0.061
Average training loss at batch  3000 : 0.055

Average training loss after epoch  1 : 0.074
Average validation loss after epoch  1 : 0.075
2019-12-05 00:56:20.976297 | Epoch 2


Average training loss at batch  0 : 0.043
Average training loss at batch  1000 : 0.041
Average training loss at batch  2000 : 0.056
Average training loss at batch  3000 : 0.040

Average training loss after epoch  2 : 0.045
Average validation loss after epoch  2 : 0.058
2019-12-05 00:59:25.675293 | Epoch 3


Average training loss at batch  0 : 0.032
Average training loss at batch  1000 : 0.034
Average training loss at batch  2000 : 0.040
Average training loss at batch  3000 : 0.030

Average training loss after epoch  3 : 0.033
Average validation loss after epoch  3 : 0.052
2019-12-05 01:02:30.942499 | Epoch 4


...

Average training loss at batch  0 : 0.006
Average training loss at batch  1000 : 0.006
Average training loss at batch  2000 : 0.008
Average training loss at batch  3000 : 0.008

Average training loss after epoch  7 : 0.009
Average validation loss after epoch  7 : 0.062
2019-12-05 02:46:24.206630 | Epoch 8


Average training loss at batch  0 : 0.005
Average training loss at batch  1000 : 0.010
Average training loss at batch  2000 : 0.008
Average training loss at batch  3000 : 0.007

Average training loss after epoch  8 : 0.007
Average validation loss after epoch  8 : 0.059
2019-12-05 02:49:32.862474 | Epoch 9


Average training loss at batch  0 : 0.007
Average training loss at batch  1000 : 0.006
Average training loss at batch  2000 : 0.052
Average training loss at batch  3000 : 0.005

Average training loss after epoch  9 : 0.006
Average validation loss after epoch  9 : 0.057
/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/emb_dim=512,upweight=1/
2019-12-05 02:52:44.667335 | Epoch 0


Average training loss at batch  0 : 6.285


KeyboardInterrupt: 

In [15]:
#baseline_model, baseline_centroids, baseline_train_losses, baseline_val_losses = train_model(model, centroids, criterion, optimizer, train_loader, val_loader, num_epochs=10, path_to_save=model_dir)


## Evaluation

In [29]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

In [30]:
## This cell will change for each model
model_folder = 'baseline_randomized_embeddings/'

criterion = KMeansCriterion()
criterion = criterion.to(current_device)

#load model
path = os.getcwd()
model_dir = path + '/models/' + model_folder

opts = torch.load(model_dir+'opts')
model = neuralNetBow(opts['vocab_size'], opts['emb_dim']) #change according to model inputs
model.load_state_dict(torch.load(model_dir+'model_dict.pt',map_location=lambda storage, loc: storage))
model = model.to(current_device)
centroids = torch.load(model_dir+'centroids',map_location=lambda storage, loc: storage)

In [31]:
TP_cluster, FP_cluster=evaluation.main(model, centroids, val_loader, criterion, data_dir, current_device)

Total examples in val loader: 455
Assigned to cluster 1: 227
TP_rate: 0.9779735682819384
FP_rate: 0.022026431718061675
FN_rate: 0.7763157894736842
TN_rate: 0.2236842105263158


Accuracy: 0.6008288894041272
Precision: 0.9779735682819384
Recall: 0.5574756319180573
F1 score: 0.7101458425405646


In [34]:
TP_cluster[TP_cluster.original == 0]

,review,index,flagged_word,assignment,original
35,strong ability to execute feedback with great artistic vision .,6,great,1,0
38,listened and created a great looking logo for us .,4,great,1,0
97,james did really great and went to great lengths to get it right for me .,7,great,1,0
212,great work .,0,great,1,0
339,great and fast solution for my posgresql question,0,great,1,0


In [35]:
FP_cluster[FP_cluster.original == 0]

,review,index,flagged_word,assignment,original
3,"working with ossobüko visual was easy , all my vague requests and comments were immediately taken into account .",9,vague,0,0
6,"great professionalism by this company , excellent project management skills",1,professionalism,0,0
18,it 's a pleasure to collaborate with him !,5,collaborate,0,0
19,wilndr understood my description to communicate my brand in the logo design .,5,communicate,0,0
27,"fantastic designer , easy to work with , made changes to models without complaint .",11,models,0,0
30,excellent experience !,1,experience,0,0
51,i have enjoyed working with chris and he has helped me to implement and strategize difficult and challenging situations .,15,difficult,0,0
67,nice results !,0,nice,0,0
68,"the first submission i received was the eventual winner , with only minor changes suggested by me .",8,winner,0,0
69,feel confident in choosing freshinnet ! !,0,feel,0,0
